# API Request for Connection to Sentinel Hub 

## Sentinel 1AB

## Import Dependencies

In [28]:
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

##### Note: Following registration to Sentinel hub you can use the dashboard to get user specific client_id and client_secret.

In [29]:
client_id = ''
client_secret = ''

In [30]:
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

In [31]:
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token', client_id=client_id,client_secret=client_secret)

In [32]:
resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

###  Two-month Temporal Average on S1GRD Orthorectified Gamma0 in VV polarization

In [33]:
url = "https://services.sentinel-hub.com/api/v1/process" 
headers = {
    "Authorization": f"Bearer {token['access_token']}"
}
evalscript = """
//VERSION=3

function setup() {
  return {
    input: ["VV", "dataMask"],
    output: {id:"default", bands: 1},
    mosaicking: Mosaicking.ORBIT
    }
}

function evaluatePixel(samples) {
  return [calculateAverage(samples)]
}

function calculateAverage(samples) {
  var sum = 0;
  var nValid = 0;
  for (let i = 0; i < samples.length; i++) {
    var sample = samples[i];
    if (sample.dataMask != 0) {
      nValid++;
      sum += toDb(sample.VV);
    }
  }
  return sum / nValid;
}

function toDb(linear) {
  return Math.max(0, Math.log(linear) * 0.21714724095 + 1)
}
""".rstrip()

data = {
    "input": {
        "bounds":{
            "bbox":[
                1360000,5121900,1370000,5131900
            ],
            "properties":{
                "crs":"http://www.opengis.net/def/crs/EPSG/0/3857"
            }
        },
        "data": [
            {
                "type":"S1GRD",
                "dataFilter":{
                    "timeRange":{
                        "from":"2020-02-01T00:00:00Z",
                        "to":"2020-04-02T23:59:59Z"
                    },
                    "orbitDirection": "ASCENDING"
                },
                "processing":{
                    "orthorectify": "true"
                }
            }
        ]
    },
    "output": {
        "width":512,
        "height":512,
        "response":[
            {
                "identifier": "default",
                "format":{
                    "type":"image/png"
                }
            }
        ]
    },
    "evalscript": evalscript
}
response = requests.post(url, json=data, headers=headers)
print(response)

<Response [200]>


In [34]:
if response.status_code == 200:
    with open('TemporalAverage_S1_VV2.tif', 'wb') as f:
        for chunk in response:
            f.write(chunk)